In [24]:
# -*- coding: utf-8 -*-
# 北京市个税 + 社保公积金（基数35811）的年度税后收入计算
# 年终奖单独计税

# ----------------------------
# 个税速算表（中国大陆）
# ----------------------------
TAX_BRACKETS = [
    (36000, 0.03, 0),
    (144000, 0.10, 2520),
    (300000, 0.20, 16920),
    (420000, 0.25, 31920),
    (660000, 0.30, 52920),
    (960000, 0.35, 85920),
    (float('inf'), 0.45, 181920),
]

#----------------------------
#全年奖金一次性扣除速算表
#----------------------------
BONUS_BRACKETS = [
    (3000, 0.03, 0),
    (12000, 0.10, 210),
    (25000, 0.20, 1410),
    (35000, 0.25, 2660),
    (55000, 0.30, 4410),
    (80000, 0.35, 7160),
    (float('inf'), 0.45, 15160),
]

SPECIAL_DEDUCTION = 1500      # 每月专项附加扣除
SPECIAL_YEAR = SPECIAL_DEDUCTION * 12  # 全年 18000
BASE_CAP = 35811              # 北京 2024 社保缴费基数封顶
GJJ_RATE = 0.12               # 公积金个人缴费比例

def calc_tax(income):
    """累计收入计算年税"""
    for up, rate, quick in TAX_BRACKETS:
        if income <= up:
            return income * rate - quick
    return 0


def annual_bonus_tax(bonus):
    """年终奖单独计税"""
    avg = bonus / 12
    for up, rate, quick in BONUS_BRACKETS:
        if avg <= up:
            return bonus * rate - quick
    return 0


def annual_salary_after_tax(monthly_salary, bonus=0):
    """
    计算年度税后收入：
    - 月工资 < 35811 → 按工资缴费
    - 月工资 >= 35811 → 按 35811 封顶基数缴费
    """

    # ----------------------------------------
    # 五险一金（以 min(月工资, 封顶基数) 为基数）
    # ----------------------------------------
    base = min(monthly_salary, BASE_CAP)

    pension = base * 0.08           # 养老
    medical = base * 0.02 + 3       # 医疗 + 大病统筹3元
    unemployment = base * 0.005     # 失业
    gjj = base * GJJ_RATE           # 公积金

    insurances = pension + medical + unemployment + gjj  # 个人缴纳总额

    # ----------------------------------------
    # 应纳税所得额（含专项附加扣除）
    # ----------------------------------------
    tax_standard = 5000  # 起征点
    taxable_month = monthly_salary - insurances - tax_standard - SPECIAL_DEDUCTION
    taxable_month = max(taxable_month, 0)

    # 年度工资累计应税额
    annual_taxable = taxable_month * 12

    # 年度个税（不含年终奖）
    annual_tax = calc_tax(annual_taxable)

    # ----------------------------------------
    # 年终奖税
    # ----------------------------------------
    bonus_tax = annual_bonus_tax(bonus)

    # ----------------------------------------
    # 年度税后收入 = 工资收入 + 奖金 - 税 - 五险一金
    # ----------------------------------------
    net_year = (
        monthly_salary * 12
        + bonus
        - annual_tax
        - bonus_tax
        - insurances * 12
    )

    return {
        "缴费基数": base,
        "公积金": gjj,
        "五险一金(月)": insurances,
        "全年五险一金": insurances * 12,
        "全年工资个税": annual_tax,
        "年终奖个税": bonus_tax,
        "年度税后收入": net_year
    }



rows = []
for monthly_salary in range(30000, 80001, 5000):
    bonus = monthly_salary * 3
    res = annual_salary_after_tax(monthly_salary, bonus=bonus)
    row = {
        "税前月工资": monthly_salary,
        "税前年度工资": monthly_salary * 12 + bonus,
        "税后年度收入(含年终)": res["年度税后收入"],
        "税后年度收入(含公积金)": res["年度税后收入"] + res["公积金"] * 12 *2,
        "税后年度年终奖": bonus - res["年终奖个税"],
        "当年个税总计": res["全年工资个税"] + res["年终奖个税"]
    }
    rows.append(row)

df = pd.DataFrame(rows)
df


,税前月工资,税前年度工资,税后年度收入(含年终),税后年度收入(含公积金),税后年度年终奖,当年个税总计
0,30000,450000,336901.200,423301.200,81210.0,32062.800
1,35000,525000,387601.200,488401.200,94710.0,42862.800
2,40000,600000,447085.725,550221.405,108210.0,56188.575
3,45000,675000,505585.725,608721.405,121710.0,72688.575
4,50000,750000,550022.010,653157.690,121410.0,103252.290
5,55000,825000,604022.010,707157.690,133410.0,124252.290
6,60000,900000,658022.010,761157.690,145410.0,145252.290
7,65000,975000,712022.010,815157.690,157410.0,166252.290
8,70000,1050000,765758.295,868893.975,169410.0,187516.005
9,75000,1125000,816758.295,919893.975,181410.0,211516.005


In [25]:
#增加一列显示年度增加的工资，按列的逻辑
df['年度增加工资'] = df['税前年度工资'].diff().fillna(0)
#增加一列显示年度增加的税后收入，按列逻辑
df['年度增加税后收入'] = df['税后年度收入(含公积金)'].diff().fillna(0)


In [27]:
df = df.astype(int)
df

,税前月工资,税前年度工资,税后年度收入(含年终),税后年度收入(含公积金),税后年度年终奖,当年个税总计,年度增加工资,年度增加税后收入
0,30000,450000,336901,423301,81210,32062,0,0
1,35000,525000,387601,488401,94710,42862,75000,65100
2,40000,600000,447085,550221,108210,56188,75000,61820
3,45000,675000,505585,608721,121710,72688,75000,58500
4,50000,750000,550022,653157,121410,103252,75000,44436
5,55000,825000,604022,707157,133410,124252,75000,54000
6,60000,900000,658022,761157,145410,145252,75000,54000
7,65000,975000,712022,815157,157410,166252,75000,54000
8,70000,1050000,765758,868893,169410,187516,75000,53736
9,75000,1125000,816758,919893,181410,211516,75000,51000
